# Naive Bayes Document Classification

We must compute several values, 

Priors:

$P(c)=\frac{N_c}{N}$
where $N_c$ is just number of documents with class and $N$ number of documents

We will calculate the conditional probabilities of each word in the document. For the purposes of this calculation we will not calculate conditional probabilities for every single word, but only the words in D1 and D2

Using $$P(w|c)=\frac{count(w,c)+\lambda}{count(c)+|V|\cdot \lambda}$$
Using $\lambda = 0.1$
Example calculation:


$P(rose|vegetable)=\frac{0+0.1}{8+7\cdot 0.1}$
Other calculations outlined below

We then find the maximum probablity of a document being in a class by using
Where $c$ is class and $d$ document
$P(c|d)=P(c) \cdot \prod_i^n{P(d_i|c)}$

Example calculation:
$P(flower|D1)=P(flower) \cdot P(rose|flower) \cdot P(lily|flower) \cdot P(apple|flower) \cdot P(carrot|flower)$



In [1]:
def p(wc, c, v, l=0.1):
    return (wc + l)/(c + v * l)

P={}

P[('rose', 'vegetable')] = p(0, 8, 7)
P[('lily', 'vegetable')] = p(0, 8, 2)
P[('apple', 'vegetable')] = p(0, 8, 2)
P[('carrot', 'vegetable')] = p(1, 8, 2)

P[('rose', 'flower')] = p(6, 13, 7)
P[('lily', 'flower')] = p(1, 13, 2)
P[('apple', 'flower')] = p(0, 13, 2)
P[('carrot', 'flower')] = p(0, 13, 2)

P[('rose', 'fruit')] = p(1, 14, 7)
P[('lily', 'fruit')] = p(1, 14, 2)
P[('apple', 'fruit')] = p(2, 14, 2)
P[('carrot', 'fruit')] = p(1, 14, 2)

#Priors
P['vegetable'] = 1/4
P['flower'] = 3/8
P['fruit'] = 3/8

D1_flower = P['flower']*P[('rose', 'flower')]*P[('lily', 'flower')]*P[('apple', 'flower')]*P[('carrot', 'flower')]
print("D1_flower", D1_flower)
D1_fruit = P['fruit']*P[('rose', 'fruit')]*P[('lily', 'fruit')]*P[('apple', 'fruit')]*P[('carrot', 'fruit')]
print("D1_fruit", D1_fruit)
D1_vegetable = P['vegetable']*P[('rose', 'vegetable')]*P[('lily', 'vegetable')]*P[('apple', 'vegetable')]*P[('carrot', 'vegetable')]
print("D1_vegetable", D1_vegetable)

D1_flower 7.985671244629444e-07
D1_fruit 2.490268929586247e-05
D1_vegetable 5.732867232465228e-08


We take the argmax of these values and find that the fruit class is the most probable.

Similarly for D2

In [2]:
P[('pea', 'vegetable')] = p(2, 8, 3)
P[('lotus', 'vegetable')] = p(1, 8, 2)
P[('grape', 'vegetable')] = p(0, 8, 2)

P[('pea', 'flower')] = p(1, 13, 3)
P[('lotus', 'flower')] = p(0, 13, 2)
P[('grape', 'flower')] = p(0, 13, 2)

P[('pea', 'fruit')] = p(0, 14, 3)
P[('lotus', 'fruit')] = p(1, 14, 2)
P[('grape', 'fruit')] = p(2, 14, 2)

D2_flower = P['flower']*(P[('pea', 'flower')]**2)*P[('lotus', 'flower')]*P[('grape', 'flower')]
print("D2_flower", D2_flower)
D2_fruit = P['fruit']*(P[('pea', 'fruit')]**2)*P[('lotus', 'fruit')]*P[('grape', 'fruit')]
print("D2_fruit", D2_fruit)
D2_vegetable = P['vegetable']*(P[('pea', 'vegetable')]**2)*P[('lotus', 'vegetable')]*P[('grape', 'vegetable')]
print("D2_vegetable", D2_vegetable)

D2_flower 1.47219552641001e-07
D2_fruit 2.1008472857159783e-07
D2_vegetable 2.618107011591733e-05


We find that D2 is classed as vegetable

# Word Sense Disambiguation

Counting all the senses will be done by putting each word through wordnet

In the cold weather, they started to the city. They were least worried protecting themselves
against the common cold. After she signed the agreement, a cold chill crept up her spine.
“Chill, its not that serious,” her husband assured and left to deposit cash at the bank.



In [3]:
from nltk import download
download('wordnet')
download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cdilg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cdilg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk.corpus import wordnet as wn
import numpy as np
import string

raw = "In the cold weather, they started to the city. They were least worried protecting themselves against the common cold. After she signed the agreement, a cold chill crept up her spine. Chill, its not that serious, her husband assured and left to deposit cash at the bank"
sents = [s.translate(str.maketrans('','', string.punctuation)).lower() for s in raw.strip().split(".")]

sentence_senses = []
word_senses = {}
for s in sents:
    sentencecount = 0
    for word in s.split(' '):
        syns = max([len(wn.synsets(word)), 1])
        sentencecount *= syns
        word_senses[word] = syns
    sentence_senses += [sentencecount]
    
print("Total senses: ", np.product(np.array(sentence_senses)))
print("Distinct combinations of senses: ", sentence_senses)
#print(word_senses)


Total senses:  0
Distinct combinations of senses:  [0, 0, 0, 0]


# This is not working
# please investigate
# seems to be something to do with how the lists are being accessed



Language Modelling

Implement a 4 gram language model

In [77]:
from os import listdir
from nltk import word_tokenize
import pickle
from collections import Counter

def save(corpus, file):
    with open(file, 'wb') as f:
        pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

def read(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

def read_dir(directory, cachefile):
    try:
        text = read(cachefile)
    except(FileNotFoundError):
        corpus = ""
        base = directory
        for file in listdir(base):
            for line in open(base + "/" + file):
                corpus += ' ' + line.strip().lower().replace('  ', ' ')
        text = word_tokenize(corpus) 
        save(text, cachefile)
    finally:
        return text

# we need to remove words that occur less than 5 times and replace with UNK
# count the items in the list. Figure out which ones are greater 
# unkwords = [w for w in [w for w in wordcount.keys() if wordcount[w] <= unk_threshold]]

# we want a list of indices for which to replace with 'UNK'
# go through the list, keep an index of where each word ocurrs. 
# at the end, count all of the lengths of these lists
# for each list which is less than 5. go to the text list and replace each element with 'UNK'

def replace_unk(text, threshold, savefile):
    try:
        return read(savefile)
    except(FileNotFoundError):
        counterdict = {}
        for i, t in enumerate(text):
            if t in counterdict.keys():
                counterdict[t].append(i)
            else:
                counterdict[t] = [i]

        for locations in counterdict:
            #print(locations, len(counterdict[locations]))
            if len(counterdict[locations]) <= threshold:
                for loc in counterdict[locations]:
                    text[loc] = 'UNK'
        save(text, savefile)
        return text

#find out the definition of 4 gram counts
#probably count all of the ways 3 previous words occur
#make a big table

def ngram(n, text, outfile):
    ngrams = {}
    for i in range(n, len(text)+1):
        #get the previous n words.
        gram = tuple(text[i-n:i])
        if gram in ngrams.keys():
            ngrams[gram] += 1
        else:
             ngrams[gram] = 1
    #save a textual representation of the dict to file
    
    with open(outfile, 'w') as f:
        for line in sorted(ngrams, key=ngrams.get, reverse=True):
            f.write(' '.join(line) + ' ' + str(ngrams[line]) + '\n')
    return ngrams



In [78]:
text = read_dir('gutenberg', 'gutenberg-corpus.txt')
text = replace_unk(text, 5, 'gutenberg-unk.txt')
guten4 = ngram(4, text, 'gutenberg-4grams.txt')
guten3 = ngram(3, text, 'gutenberg-3grams.txt')

Perplexity formula: $PP(W)=\left(\prod_{i=1}^N{\frac{1}{P(w|w_{n-1}, w_{n-2}, w_{n-3}, w_{n-4})}}\right)^\frac{1}{N}$


Probability of words: $P(w)=\frac{num\space counts}{total \space words}$
$P(w_n|w_{n-1}, w_{n-2}, w_{n-3}, w_{n-4})=\frac{C(w_{n-4}w_{n-3}w_{n-2}w_{n-1}w)+0.1}{C(w_{n-4}w_{n-3}w_{n-2}w_{n-1})+|V|\times 0.1}$

In [79]:
imdb = read_dir('imdb_data', 'imdb-corpus.txt')
imdb = replace_unk(imdb, 5, 'imdb-unk.txt')
#imdbmodel = ngram(4, imdb, 'imdb-ngrams.txt')
#imdbmodel3 = ngram(3, imdb, 'imdb-3grams.txt')
import math

def calculate_probability(model, onelessmodel, word, context, n = 4, lam = 0.1):
    #for each of the words, we need the prior 4 words. We will look this up and find whether we have a match
    try:
        ret = (model[(context[0], context[1], context[2], word)] + lam)/(onelessmodel[(context[0], context[1], context[2])]+len(model)*lam)
    except:
        ret = lam/(len(model)*lam)
    #here v is a vocabulary of n-grams, so will be the count of the ngrams
    
    return ret

        
def perplexity(model, onelessmodel, text, n = 4):
    #this takes in text, which the existing probabilities and counts are used to 
    #come up with a number, all of the probabilities multiplied together. We probably can use log for this?
    #Perplexity is a measure of how probable the model is at generating a sentence
    #
    #Perplexity is an integer - lower better
    pp = 1
    V = len(model)
    for i, word in enumerate(text):
        #calculate the probability of this word
        #TODO implement log sum instead
        pp *= 1/calculate_probability(model, onelessmodel, word, text[i-n:i-1])
    return math.pow(pp, 1/V)

def wikiperplexity(model, onelessmodel, text, n=4):
    pp = 0
    V = len(model)
    for i, word in enumerate(text):
        pp -= math.log(2, calculate_probability(model, onelessmodel, word, text[i-n:i-1]))
    return math.pow(2, pp/V)
#Currently, this works: 
#print(guten4[('the', 'children', 'of', 'israel')])

print(calculate_probability(guten4, guten3, 'israel', ['the', 'children', 'of']))
print(calculate_probability(guten4, guten3, 'were', ['children', 'of', 'israel']))
print(wikiperplexity(guten4, guten3, ['the', 'children', 'of', 'israel', 'were', 'off', 'on', 'the', 'beaches', 'when']))
print(perplexity(guten4, guten3, ['the', 'children', 'of', 'israel', 'were', 'off', 'on', 'the', 'beaches', 'when']))
#perplexity(guten4, guten3, ["the", "children", "of", "israel", "are", "well"])

0.0030798517076259754
7.797958291094393e-05
1.0000001605774118
1.000070634697219


In [80]:
news = read_dir('news_data', 'news-corpus.txt')
news = replace_unk(news, 5, 'news-unk.txt')
news4 = ngram(4, news, 'news-4grams.txt')
news3 = ngram(3, news, 'news-3grams.txt')

print(wikiperplexity(news4, news3, ['the', 'children', 'of', 'israel', 'were', 'off', 'on', 'the', 'beaches', 'when']))
print(perplexity(news4, news3, ['the', 'children', 'of', 'israel', 'were', 'off', 'on', 'the', 'beaches', 'when']))

1.0000033056111426
1.0009467110114756


## POS Tagging HMM

First find the tag unigram and tag bigram counts from the corpus



In [94]:
import operator
import random
#read in the file/s?
import nltk
from nltk.corpus import brown

#nltk.download('brown')
def read_brown(directory, cachefile):
    try:
        text = read(cachefile)
    except(FileNotFoundError):
        corpus = []
        base = directory
        for file in listdir(base):
            #print(file)
            for sent in open(base + "/" + file):
                if sent == "\n": continue
                wordlist = []
                for word in sent.strip().split(' '):
                    #split the word and it's tag
                    if word == '': 
                        continue
                    wordlist.append(word.split('/'))
                    
                corpus += [['<s>', '<s>']] + wordlist + [['</s>', '</s>']]
        text = corpus
        save(text, cachefile)
    finally:
        return text
brown = read_brown('brown', 'brown-cache.txt')

#calculate the word-tag counts
#lets do this in the same dictionary way we did earlier

def wordtag(text, outfile):
    pairs = {}
    for word in text:
        #ignore the sentence tags
        if (word == ['<s>', '<s>'] or word == ['</s>', '</s>']): continue
        tagpair = tuple(word)
        if tagpair in pairs.keys():
            pairs[tagpair] += 1
        else:
             pairs[tagpair] = 1
    #save a textual representation of the dict to file
    
    with open(outfile, 'w') as f:
        for word in sorted(pairs, key=pairs.get, reverse=True):
            f.write(' '.join(word) + ' ' + str(pairs[word]) + '\n')
    return pairs

def tagunigram(text, outfile):
    '''This is literally a unigram of the tag, t_n. That is we 
    will not consider the word association and will instead just
    consider the impact of the counts of tags themselves.'''
    unigrams = {}
    for word in text:
        #ignore the sentence tags
        if (word == ['<s>', '<s>'] or word == ['</s>', '</s>']): continue 
        tag = word[1]
        if tag in unigrams.keys():
            unigrams[tag] += 1
        else:
             unigrams[tag] = 1
    #save a textual representation of the dict to file
    
    with open(outfile, 'w') as f:
        #TODO there is a problem with the way this joins - it's assuming a tuple
        for word in sorted(unigrams, key=unigrams.get, reverse=True):
            f.write(' '.join(word) + ' ' + str(unigrams[word]) + '\n')
    return unigrams
    
def savecounts(d, file):
    with open(file, 'w') as f:
        for token in sorted(d, key=d.get, reverse=True):
            f.write(' '.join(token) + ' ' + str(d[token]) + '\n')
            
def tagbigram(text, outfile):
    '''Here we consider both t_n and t_n-1 and report the counts. 
    Again we do not stop to consider the effects of the word association'''
    bigrams = {}
    for i in range(len(text)):
        #ignore the sentence tags
        if (text[i] == ['<s>', '<s>'] or text[i] == ['</s>', '</s>']): continue
        t = text[i][1]
        t1 = text[i-1][1]
        if (t, t1) in bigrams.keys():
            bigrams[(t, t1)] += 1
        else:
             bigrams[(t, t1)] = 1
    savecounts(bigrams, outfile)
    return bigrams
    #save a textual representation of the dict to file


def transition(bigramtags, unigramtags):
    probabilities = {}
    for bigram in bigramtags.keys():
        probabilities[bigram] = bigramtags[bigram]/unigramtags[bigram[0]]
    savecounts(probabilities, 'brownmeta/transition-probabilities.txt')
    return probabilities

def emission(wordtags, unigramtags):
    emissionprob = {}
    for wordpair in wordtags.keys():
        emissionprob[wordpair] = wordtags[wordpair]/unigramtags[wordpair[1]]
    savecounts(emissionprob, 'brownmeta/emission-probabilities.txt')
    return emissionprob

class postagger:
    
    def __init__(self, wordtags, unigramprobabilities, bigramprobabilities):
        self.wt = wordtags
        self.up = unigramprobabilities
        self.bp = bigramprobabilities
    
    def nextword(self, dct):
        #select a next item based on a random number which is weighted by the probabilities
        
        #sum all of the probabilities and normalize
        tot = 0
        for value in dct.keys():
            tot += dct[value]
        normalised = {}
        index = random.random()
        for pair in dct.keys():
            index -= dct[pair] / tot
            if index <= 0.0:
                return pair
        
    def predictSentence(self):
        #lets just feed in a start tag, 
        #pick the highest probability next tag, and then 
        #the highest probability word of that tag
        
        #find all things with a start tag at the start
        
        sent = []
        humansent = []
        priortag = '<s>'
        for i in range(25):
            subset = {}
            for tags in self.bp.keys():
                if priortag == tags[1]:
                    subset[tags] = self.bp[tags]
            selectedbigram = self.nextword(subset)
            
            print(selectedbigram)
            
            currenttag = selectedbigram[0]
            
            potentialwordtags = {}
            
            for wordtag in self.wt.keys():
                if currenttag == wordtag[1]:
                    potentialwordtags[wordtag] = self.wt[wordtag]
            currentword = self.nextword(potentialwordtags)
            
            print(currentword)
            
            sent.append('/'.join(currentword))
            humansent.append(currentword[0])
            priortag = currenttag
        print(' '.join(sent))
        print(' '.join(humansent))
    
    
tags = wordtag(brown, 'brownmeta/brownwordtag.txt')
unitags = tagunigram(brown, 'brownmeta/brownuni.txt')
bitags = tagbigram(brown, 'brownmeta/brownbigrams.txt')

transitionprobs = transition(bitags, unitags)
emissionprobs = emission(tags, unitags)

pos = postagger(tags, unitags, bitags)
#for i in range(1000):
#    print(pos.nextword({('flog', 'np'): 0.5, ('dog', 'nn'):0.9, ('whack', 'dd'):0.01}))
pos.predictSentence()


('np', '<s>')
('Mickey', 'np')
(',', 'np')
(',', ',')
('bez*', ',')
("isn't", 'bez*')
('jj', 'bez*')
('bad', 'jj')
('jj', 'jj')
('high', 'jj')
('nn', 'jj')
('loot', 'nn')
('in', 'nn')
('regarding', 'in')
('cd', 'in')
('6-12', 'cd')
('nns', 'cd')
('preparations', 'nns')
('cc', 'nns')
('and', 'cc')
('at', 'cc')
('the', 'at')
('jj', 'at')
('old', 'jj')
('nns', 'jj')
('doors', 'nns')
("''", 'nns')
("''", "''")
(',', "''")
(',', ',')
('cc', ',')
('or', 'cc')
('jj', 'cc')
('anionic', 'jj')
('--', 'jj')
('--', '--')
('in', '--')
('in', 'in')
('ppo', 'in')
('her', 'ppo')
('.', 'ppo')
('.', '.')
('.', '.')
('!', '.')
('.', '.')
('!', '.')
('.', '.')
('.', '.')
('.', '.')
('.', '.')
Mickey/np ,/, isn't/bez* bad/jj high/jj loot/nn regarding/in 6-12/cd preparations/nns and/cc the/at old/jj doors/nns ''/'' ,/, or/cc anionic/jj --/-- in/in her/ppo ./. !/. !/. ./. ./.
Mickey , isn't bad high loot regarding 6-12 preparations and the old doors '' , or anionic -- in her . ! ! . .
